In [85]:
import numpy as np
import pandas as pd
import torch.nn as nn
import h5py
import pickle
import torch
import time
import yaml
import copy
import math
import traceback
import sys
import time
import os
import torch.nn.functional as F
import torchvision
import transformers
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

file_dir = '/home/quang/Documents/XAI_env-main/data/processed/IMS/'
file_name = 'x_train.hdf5'

with h5py.File(file_dir + file_name, 'r') as file:
    dataset1 = file['x_train'][:]  # Replace 'dataset_name' with the actual dataset key or name
    dataset1_shape = dataset1.shape
    a = dataset1.dtype
print(dataset1_shape)

   

(7304, 20)


In [86]:
import h5py

file_dir = '/home/quang/Documents/XAI_env-main/data/processed/IMS/'
file_name = 'y_train.hdf5'

with h5py.File(file_dir + file_name, 'r') as file:
    dataset2 = file['y_train'][:]  # Replace 'dataset_name' with the actual dataset key or name
    dataset2_shape = dataset2.shape

print(dataset2_shape)

(7304, 3)


In [114]:
class CustomDataset:
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        current_sample = torch.tensor(self.data[idx, :], dtype=torch.float)
        current_target = torch.tensor(self.targets[idx], dtype=torch.long)
        data_shape = current_sample.shape[1:]
        return {
            "sample": current_sample,
            "target": current_target,
        }

In [88]:
data = dataset1
data.shape

(7304, 20)

In [89]:
targets = dataset2
targets.shape

(7304, 3)

In [90]:
custom_dataset1 = CustomDataset(data=data,targets=targets)

In [91]:
len(custom_dataset1)

7304

In [92]:
print(custom_dataset1)

In [93]:
custom_dataset1[0]

{'sample': tensor([0.1569, 0.0183, 0.0194, 0.0166, 0.1277, 0.0036, 0.0034, 0.0016, 0.0035,
         0.0034, 0.0028, 0.0036, 0.0047, 0.0036, 0.0046, 0.0066, 0.0068, 0.0120,
         0.0117, 0.0073]),
 'target': tensor([34,  0, 10])}

In [94]:
train_loader = DataLoader(custom_dataset1,
                          batch_size=128,
                          shuffle=True)

In [95]:
for i,data in train_loader:
    print(i)

sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample
sample


In [115]:
for i, data in enumerate(train_loader):
    print(i)
    print(data)

0
{'sample': tensor([[0.1333, 0.0093, 0.0269,  ..., 0.0132, 0.0079, 0.0060],
        [0.0716, 0.0133, 0.0227,  ..., 0.0094, 0.0079, 0.0060],
        [0.0537, 0.0079, 0.0154,  ..., 0.0293, 0.0232, 0.0150],
        ...,
        [0.0603, 0.0204, 0.0169,  ..., 0.0048, 0.0077, 0.0026],
        [0.0841, 0.0195, 0.0244,  ..., 0.0065, 0.0128, 0.0061],
        [0.0505, 0.0127, 0.0198,  ..., 0.0086, 0.0083, 0.0057]]), 'target': tensor([[22,  0, 21],
        [36,  0,  8],
        [ 2,  0,  4],
        [ 8,  0, 36],
        [21,  0, 23],
        [14,  0, 30],
        [35,  0,  8],
        [31,  0, 12],
        [18,  0, 25],
        [ 6,  0,  0],
        [ 1,  0,  5],
        [21,  0, 23],
        [26,  0, 18],
        [35,  0,  8],
        [ 2,  0,  4],
        [28,  0, 16],
        [10,  0, 34],
        [31,  0, 13],
        [29,  0, 15],
        [21,  0, 23],
        [21,  0, 22],
        [27,  0, 17],
        [ 6,  0, 38],
        [44,  0,  0],
        [ 1,  0,  5],
        [42,  0,  2],
      

{'sample': tensor([[0.1071, 0.0133, 0.0280,  ..., 0.0107, 0.0101, 0.0104],
        [0.1060, 0.0241, 0.0270,  ..., 0.0103, 0.0126, 0.0094],
        [0.0748, 0.0126, 0.0242,  ..., 0.0060, 0.0061, 0.0058],
        ...,
        [0.0757, 0.0149, 0.0274,  ..., 0.0083, 0.0109, 0.0083],
        [0.1656, 0.0132, 0.0217,  ..., 0.0119, 0.0121, 0.0060],
        [0.0788, 0.0075, 0.0240,  ..., 0.0071, 0.0079, 0.0040]]), 'target': tensor([[38,  0,  6],
        [32,  0, 11],
        [ 7,  0, 37],
        [ 8,  0, 36],
        [ 1,  0,  5],
        [18,  0, 25],
        [27,  0, 16],
        [26,  0, 17],
        [10,  0, 34],
        [ 3,  0, 40],
        [42,  0,  2],
        [23,  0, 21],
        [ 4,  0,  1],
        [15,  0, 29],
        [ 6,  0,  0],
        [27,  0, 17],
        [30,  0, 13],
        [33,  0, 10],
        [ 5,  0,  1],
        [ 5,  0, 39],
        [18,  0, 26],
        [34,  0, 10],
        [43,  0,  0],
        [40,  0,  3],
        [21,  0, 23],
        [15,  0, 29],
        

In [103]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_dim,embed_dim, noise_level):
        super(Autoencoder, self).__init__()
        self.input_size, self.hidden_dim, self.noise_level = input_size, embed_dim,noise_level
        self.embed_dim = embed_dim
        self.fc1 = nn.Linear(self.input_size, self.hidden_dim)
        self.fc2 = nn.Linear(self.hidden_dim, self.input_size)
        
    def encoder(self,x):
        x = self.fc1(x)
        h1 = F.relu(x)
        return h1
    
    def mask(self,x):
        corrupted_x = x + self.noise_level + torch.randn_like(x)   # randn_like  Initializes a tensor where all the elements are sampled from a normal distribution.
        return corrupted_x
    
    def decoder(self, x):
        h2 = self.fc2(x)
        return h2
    
    def forward (self, x):
        out = self.mask(x) # Adding noise to feed the network
        encoder = self.encoder(out)
        decoder = self.decoder(encoder)
        return encoder, decoder 
    
    ## Transformer 
    ### Positional encoding
class PositionalEncoding(nn.Module):
        def __init__(self,d_model, dropout=0.0,max_len=16):
            super(PositionalEncoding, self).__init__()
            pe = torch.zeros(max_len,d_model)
            position = torch.arange(0,max_len, dtype = torch.float).unsqueeze(1)
            
            div_term = torch.exp(torch.arange(0,d_model,2).float()*(-math.log(10000.0) / d_model))
            
            pe[:, 0::2] = torch.sin(position * div_term)
            pe[:, 1::2] = torch.cos(position * div_term)

            pe = pe.unsqueeze(0).transpose(0, 1)

            self.register_buffer('pe', pe)
            
        def forward(self, x):
            x = x + self.pe[:x.size(1), :].squeeze(1)
            return x
        
class Net(nn.Module):
        def __init__(self,feature_size,num_layers,n_head,dropout,noise_level,embed_dim):
            super(Net,self).__init__()
            self.embed_dim = embed_dim
            self.hidden_dim = 4*embed_dim
            self.auto_hidden = int(feature_size / 2)
            input_size = self.auto_hidden
            self.pos = PositionalEncoding(d_model=input_size, max_len=input_size)
            encoder_layers = nn.TransformerEncoderLayer(d_model=input_size, nhead=n_head, dim_feedforward=self.hidden_dim, dropout=dropout)
            self.cell = nn.TransformerEncoder(encoder_layers,num_layers=num_layers)
            self.linear = nn.Linear(input_size,1)
            self.autoencoder = Autoencoder(input_size = feature_size, hidden_dim = self.auto_hidden,embed_dim = embed_dim, noise_level=noise_level)
              
        def forward(self,x):
            batch_size, feature_num, feature_size = x.shape
            encode, decode = self.autoencoder(x.view(batch_size,-1).float()) # Equals batch_size * seq_len
            out = encode.reshape(batch_size, -1, self.auto_hidden)
            out = self.pos(out)
            out = out.reshape(1,batch_size,-1)  #(1,batch_size,feature_size)
            out = self.cell(out)
            out = out.reshape(batch_size,-1)
            out = self.linear(out)
            
            return out, decode
        

In [109]:
model = Net(feature_size = 20,
            noise_level = 0.01,   
            num_layers=3,
            embed_dim = 16,
            n_head = 2,
            dropout=0.1)

In [110]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=2)
criterion = nn.MSELoss()

In [119]:
print(model)


Net(
  (pos): PositionalEncoding()
  (cell): TransformerEncoder(
    (layers): ModuleList(
      (0-2): 3 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=10, out_features=10, bias=True)
        )
        (linear1): Linear(in_features=10, out_features=64, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=64, out_features=10, bias=True)
        (norm1): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (linear): Linear(in_features=10, out_features=1, bias=True)
  (autoencoder): Autoencoder(
    (fc1): Linear(in_features=20, out_features=16, bias=True)
    (fc2): Linear(in_features=16, out_features=20, bias=True)
  )
)


In [117]:
num_epochs = 100
batch_size = 128
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for epoch in range(num_epochs):
    model.train()
    for i, batch in enumerate(train_loader):
        x = batch["sample"].to(device)
        rul = batch["target"].to(device)
        
        #Forward
        x = x.unsqueeze(1)
        output = model(x)
        loss = criterion(output,rul)
        
        #Backward and Optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        print("Epoch{}/{}.Iter{}/{}.Loss{}").format(epoch+1,num_epochs,iter,num_iters,loss)

RuntimeError: shape '[128, -1, 10]' is invalid for input of size 2048